If you want to be in the `PedestrianAvoidancePOMDP` environment, run the following cell. If the package has been added to the main julia environment, it is not necessary.

In [1]:
] activate ../ 

In [2]:
using Revise
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using GridInterpolations

using PedestrianAvoidancePOMDP
using POMDPPolicies

using JLD2
using FileIO
using Reel
using Random
using POMDPs
using POMDPModelTools
using LinearAlgebra

using EmergencyBrakingSystem


Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /buildworker/worker/package_linux64/build/src/locks.h:138 [inlined]
jl_typeinf_end at /buildworker/worker/package_linux64/build/src/gf.c:2568
typeinf_ext at ./compiler/typeinfer.jl:568
typeinf_ext at ./compiler/typeinfer.jl:604
jfptr_typeinf_ext_1.clone_1 at /home/xubuntu/julia-1.0.1/lib/julia/sys.so (unknown line)
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2184
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1537 [inlined]
jl_apply_with_saved_exception_state at /buildworker/worker/package_linux64/build/src/rtutils.c:257
jl_type_infer at /buildworker/worker/package_linux64/build/src/gf.c:275
jl_compile_method_internal at /buildworker/worker/package_linux64/build/src/gf.c:1786 [inlined]
jl_fptr_trampoline at /buildworker/worker/package_linux64/build/src/gf.c:1830
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2184
do_call 

In [3]:
policy = load("../policy/policy.jld2")["policy"]; # need to be updated

In [4]:
pomdp = SingleOCFPOMDP()
updater = SingleOCFUpdater(pomdp)

parameters = CrosswalkParams()
parameters.roadway_length = 400.0
parameters.obstacles_visible = true

obstacle_offset = -2
obstacle_1 = ConvexPolygon([VecE2(34, obstacle_offset), VecE2(34, obstacle_offset-3), VecE2(46.5, obstacle_offset-3), VecE2(46.5, obstacle_offset)],4)
obstacle_2 = ConvexPolygon([VecE2(34, +4.5), VecE2(34, +7.5), VecE2(46.5, +7.5), VecE2(46.5, +4.5)],4)
parameters.obstacles = [obstacle_1, obstacle_2]
#parameters.obstacles = [obstacle_2]

env = CrosswalkEnv(parameters)
pomdp.env = env


ego_id = 1
ped_id = 2
ped2_id = 3


ego_v = 14.0
ped_v = 5.0/3.6
hitpoint = 50             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 100.0
ped_y_start = -5.0
ego_y = 0.0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;

#=
if ( false )
    ego_v = 0.0
    ego_x = 20
    ped_x = 30
    ped_y_start = -2.0
    ped_v = 0.0
end

ped_y_start = -5.
ped_v = 1.0
ego_y = 0.0
=#

#ped_v = 0.0

#ped_y_start = -5.



# Car definition
ego_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
ego = Vehicle(ego_initial_state, VehicleDef(), 1)

# Pedestrian definition 
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

ped2_v = 0.0
ped2 = Vehicle(VehicleState(VecSE2(54., 5., 0.78), env.crosswalk, env.roadway, ped2_v), AutomotivePOMDPs.PEDESTRIAN_DEF, ped2_id)


scene = Scene()
push!(scene, ego)
push!(scene, ped)
push!(scene, ped2)



timestep = 0.2
timestep_pomdp = 0.2

pos_noise = 0.001
vel_noise = 0.000
false_positive_rate = 0.00
false_negative_rate = 0.00
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 


# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

pomdp.desired_velocity = ego_v
pomdp.ΔT = timestep
models[ego_id] = PedestrianAvoidancePOMDPFrenet(a=LatLonAccel(0.0, 0.0),
    env=env,
    sensor=sensor,
    obstacles=env.obstacles,
    timestep=timestep_pomdp,
    update_tick_high_level_planner = timestep_pomdp / timestep,
    pomdp=pomdp,
    policy=policy,
    updater=updater,
    ego_vehicle=ego,
    desired_velocity=ego_v,
    b=PedestrianAvoidancePOMDP.initBeliefAbsentPedestrian(pomdp, ego_y, ego_v)
)

models[ped_id] = ConstantPedestrian(v_desired=ped.state.v, dawdling_amp=0.0001) # dumb model
models[ped2_id] = ConstantPedestrian(v_desired=ped2.state.v, dawdling_amp=0.0001) # dumb model



nticks = 80
rec = SceneRecord(nticks+1, timestep)

sensor_observations = [Vehicle[]]
ego_vehicle = Vehicle[]
ego_a = Float64[]
collision = Bool[]

belief = Dict{Int64, SingleOCFBelief}[]
action_pomdp = SingleOCFAction[]

collision_rate = Float64[]  # not used
ttc = Float64[]             # not used
risk = Float64[]
emergency_brake_request = Bool[]      # not used
prediction_obstacle = Vector{Array{Float64}}()  # not used


obs_callback = (ObservationCallback(sensor_observations, ego_vehicle, 
                  ego_a, collision, belief, action_pomdp, collision_rate, ttc, 
                  risk, emergency_brake_request,prediction_obstacle),)

@elapsed begin
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
end



15.465887376

In [5]:
# Visualize scenario
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, prediction_obstacle, CarFollowCamera(2,10.0))
film = roll(render_hist, fps = fps, duration = duration)
#write("pomdp.mp4", film) # Write to a webm video

Frames{MIME{Symbol("image/png")}}("/tmp/tmpHgJA3V", 0x0000000000000050, 5.0, nothing)

In [8]:

algorithm = "PedestrianAvoidancePOMDP"
#algorithm = "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"

policy = load("../policy/policy.jld2")["policy"];

# Definition ego vehicle and pedestrian behavior

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA", "FP"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0., 10., 20., 30., 40., 50.]

# FP Tests
#CPAN25 -100, 200

ego_v = vut_speeds[9]
hit_point = hit_points[1]
scenario = scenarios[1]


### Simulate scenario with parameters above defined
# generate scenario based on scenario type
(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = PedestrianAvoidancePOMDP.generate_scenario(scenario, ego_v, hit_point)
# simulate scenario
(rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy, algorithm)
#evaluate result
(collision2, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = PedestrianAvoidancePOMDP.evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision, ped_x)



PedestrianAvoidancePOMDP
-----------------> Collision <----------------------


(true, false, 5.838888888888897, 10.486388888888886, -3.150943396226415, 30.0, -4.0)

In [9]:
# Visualize scenario
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, prediction_obstacle, CarFollowCamera(1,10.0))
film = roll(render_hist, fps = fps, duration = duration)
#write("pomdp.mp4", film) # Write to a webm video

Frames{MIME{Symbol("image/png")}}("/tmp/tmpV3VOtX", 0x0000000000000052, 20.0, nothing)

In [12]:
# Evaluate EuroNCAP scenarios

using CSV
using DataFrames


#algorithm = "EmergancyBrakingSystem"
#algorithm = "PedestrianAvoidancePOMDP"
#algorithm = "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"

policy_name = "no"

policy_name = "longitudinal_support"
#policy_name = "longitudinal_lateral_support"

policy = load("../policy/policy.jld2")["policy"];


log_filename = string("results_", algorithm, "_", policy_name, ".csv")

println(log_filename)

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA"]
vut_speeds = [50.] #[10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0.] #[0., 10., 20., 30., 40., 50.]


println("Algorithm to evaluate: ", algorithm, " Policy: ", policy_name, " (if specified)")

# Evaluate all EuroNCAP scenarios
rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, emergency_brake_request, prediction_obstacle, collision, ego_a
results = Vector[]
for scenario in scenarios
    for hit_point in hit_points
        for vut_speed in vut_speeds
            ego_v = vut_speed
            (ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = PedestrianAvoidancePOMDP.generate_scenario(scenario, ego_v, hit_point)
            if algorithm == "EmergancyBrakingSystem"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
            elseif algorithm == "PedestrianAvoidancePOMDP"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy)
            elseif algorithm == "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"
                
            else
                println("No valid algorithm defined!")
                return false
            end
            
            (collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = PedestrianAvoidancePOMDP.evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision, ped_x)
            println(collision, " ", emergency_brake_intervention, " ", dv_collision, " ", v_mean, " ", a_mean, " ", a_jerk, " ", a_min)
            result = [scenario_id, ego_v, hit_point, collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min  ]
            push!(results, result)
        end
    end 
end  


# store results from scneario evaluation in log file
df = DataFrame(results)
df = DataFrame(Matrix(df)')
rename!(df, :x1 => :scenario_id, :x2 => :ego_v, :x3 => :hit_point, :x4 => :collision, :x5 => :eb_intervention)
rename!(df, :x6 => :dv_collision, :x7 => :v_mean, :x8 => :a_mean, :x9 => :a_jerk, :x10 => :a_min)

CSV.write(log_filename, df);
df

MethodError: MethodError: no method matching iterate(::Revise.RelocatableExpr)
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:589
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:589
  iterate(!Matched::ExponentialBackOff) at error.jl:171
  ...

0

MethodError: MethodError: no method matching iterate(::Revise.RelocatableExpr)
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:589
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:589
  iterate(!Matched::ExponentialBackOff) at error.jl:171
  ...